In [2]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# Load enriched data (already cleaned and feature engineered)
df = pd.read_csv('data/cleaned/Cleaned_INDIA.csv', parse_dates=['Date'])
df.set_index('Date', inplace=True)

# Define feature columns
features = ['Close', 'MA_5', 'MA_20', 'Vol_10', 'RSI_14', 'MACD', 'Signal']

# Drop NaNs and prepare data
df.dropna(inplace=True)
X_raw = df[features].copy()
y_raw = df['Target'].copy()

In [4]:
X_scaler = MinMaxScaler()
X_scaled = X_scaler.fit_transform(X_raw)

y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y_raw.values.reshape(-1, 1))

In [5]:
def create_windows(X, y, window_size=60):
    Xs, ys = [], []
    for i in range(len(X) - window_size):
        Xs.append(X[i:i+window_size])
        ys.append(y[i+window_size])
    return np.array(Xs), np.array(ys)

X_tensor, y_tensor = create_windows(X_scaled, y_scaled.flatten())

In [6]:
def split_data(X, y, train_ratio=0.8):
    split = int(len(X) * train_ratio)
    return X[:split], X[split:], y[:split], y[split:]

X_train, X_test, y_train, y_test = split_data(X_tensor, y_tensor)

In [7]:
def build_cnn_lstm_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(units=64, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

input_shape = X_train.shape[1:]
model = build_cnn_lstm_model(input_shape)

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
36/36 [==============================] - 1s 10ms/step - loss: 0.0559 - mae: 0.1623 - val_loss: 0.0017 - val_mae: 0.0337
Epoch 2/100
36/36 [==============================] - 0s 11ms/step - loss: 0.0041 - mae: 0.0497 - val_loss: 0.0013 - val_mae: 0.0291
Epoch 3/100
36/36 [==============================] - 0s 8ms/step - loss: 0.0026 - mae: 0.0394 - val_loss: 0.0040 - val_mae: 0.0571
Epoch 4/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0023 - mae: 0.0369 - val_loss: 0.0036 - val_mae: 0.0537
Epoch 5/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0023 - mae: 0.0375 - val_loss: 0.0030 - val_mae: 0.0483
Epoch 6/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0021 - mae: 0.0357 - val_loss: 0.0031 - val_mae: 0.0484
Epoch 7/100
36/36 [==============================] - 0s 6ms/step - loss: 0.0019 - mae: 0.0341 - val_loss: 0.0033 - val_mae: 0.0507
Epoch 8/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0016 -

In [8]:
y_pred_scaled = model.predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred_scaled)
y_true = y_scaler.inverse_transform(y_test.reshape(-1, 1))

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)

print(f"Test RMSE: {rmse:.2f}")
print(f"Test MAE: {mae:.2f}")

9/9 [==============================] - 0s 2ms/step
Test RMSE: 471.12
Test MAE: 390.21


In [10]:
# -----------------------------------------------
# Step 7: Export for Power BI
# -----------------------------------------------
test_dates = df.index[-len(y_true):]

results_df = pd.DataFrame({
    'Date': test_dates,
    'Actual_Close': y_true.flatten(),
    'Predicted_Close': y_pred.flatten(),
    'Market': 'India'
})

results_df.to_csv("cnn_lstm_predictions_india.csv", index=False)
print("Exported predictions for Power BI.")


Exported predictions for Power BI.
